# AISE 3350A Project: RPS-Neural-Link
### Cyber-Physical Game Theory System
**Group Number:** 4
**Group Members:** Michael Trbovic (251358199), Murede Oluwamurede Adetiba (251372276), Tyler Lafond (251359907), William Huang (251371199)
**Date:** December 5, 2025

---

## 1. Introduction

### 1.1 Motivation & Background
Rock-Paper-Scissors-Minus-One (RPS-1) is a strategic variation of the classic intransitive hand game. Unlike standard RPS, where players throw a single hand, RPS-1 involves a two-stage decision process: players present two hands, then strategically withdraw one. This introduces a complex layer of "hand selection" where optimal play requires analyzing the opponent's available options in real-time.

In the context of **Cyber-Physical Systems (CPS)**, this project aims to build an Augmented Reality (AR) agent that bridges the physical world (human gestures) and the cyber world (game theory algorithms). Real-time analysis is critical; a delay of even a few milliseconds can render a strategic suggestion obsolete.

### 1.2 Objectives
The primary goal of the **RPS-Neural-Link** is to create a decision-support system that:
1.  **Perceives:** Uses Computer Vision to identify and classify hand gestures from a live video feed or static images.
2.  **Analyzes:** Applying Game Theory (Nash Equilibrium) and Adaptive AI (Markov Chains) to determine the optimal move.
3.  **Augments:** Projects the calculated move back to the user via a Heads-Up Display (HUD).

---

## 2. Methods

### 2.1 Computer Vision Approach
We utilized **Google MediaPipe Hands** rather than object detection models like YOLO.
* **Rationale:** RPS requires recognizing the *geometric configuration* of fingers (open vs. closed), not just the presence of a hand. MediaPipe provides 21 3D skeletal landmarks per hand, allowing for high-frequency, low-latency classification without the need for a massive labeled dataset.
* **Classification Logic:** We calculate Euclidean distances between specific landmarks (e.g., wrist-to-fingertip vs. wrist-to-knuckle) to determine if fingers are extended.
    * **Rock:** 0-1 fingers extended.
    * **Scissors:** Index & Middle fingers extended.
    * **Paper:** 4-5 fingers extended.

### 2.2 Strategic Engine (The "Cyber" Component)
Our system employs a hybrid decision engine:

1.  **The Markov Predictor (Offensive):**
    We treat the game as a sequence of events. A first-order Markov Chain records the transition probabilities of the opponent's moves (e.g., $P(Paper | Rock)$). If the opponent exhibits a predictable pattern (probability $> 33\%$ above random chance), the system predicts their next move and counters it.

2.  **Nash Equilibrium (Defensive):**
    When the opponent's behavior is random or data is insufficient, the system reverts to the Minimax principle. We calculate the payoff matrix for the current 4-hand configuration (User's 2 hands vs. Opponent's 2 hands) and determine the "Minus One" decision that statistically minimizes the maximum possible loss.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import math
from enum import Enum
from collections import Counter

# --- 1. CONFIGURATION & ENUMS ---
class HandShape(Enum):
    ROCK = 0
    PAPER = 1
    SCISSORS = 2
    UNKNOWN = -1

# --- 2. ADAPTIVE AI BRAIN ---
class MarkovPredictor:
    """
    Learns opponent patterns over time using a transition matrix.
    """
    def __init__(self):
        self.matrix = {} 
        self.last_move = None
        self.total_moves = 0 
        
    def update(self, current_move):
        self.total_moves += 1
        if self.last_move is not None:
            transition = (self.last_move, current_move)
            self.matrix[transition] = self.matrix.get(transition, 0) + 1
        self.last_move = current_move
        
    def get_prediction(self):
        # Need at least 3 moves to establish a basic pattern
        if self.total_moves < 3 or self.last_move is None:
            return None, "WAITING FOR DATA"
            
        options = [HandShape.ROCK, HandShape.PAPER, HandShape.SCISSORS]
        best_hand, highest_count = None, -1
        
        # Check which move most frequently follows the last move
        for opt in options:
            count = self.matrix.get((self.last_move, opt), 0)
            if count > highest_count:
                highest_count = count
                best_hand = opt
                
        return (best_hand, f"PREDICT: {best_hand.name}") if highest_count > 0 else (None, "UNCERTAIN")

# --- 3. GAME STRATEGY ---
class GameStrategy:
    """
    Calculates Optimal Move using Nash Equilibrium or Counter-Play.
    """
    def __init__(self):
        # Payoff Matrix: 1 = Win, -1 = Loss, 0 = Tie
        self.rules = {
            (HandShape.ROCK, HandShape.SCISSORS): 1, (HandShape.ROCK, HandShape.PAPER): -1, (HandShape.ROCK, HandShape.ROCK): 0,
            (HandShape.PAPER, HandShape.ROCK): 1, (HandShape.PAPER, HandShape.SCISSORS): -1, (HandShape.PAPER, HandShape.PAPER): 0,
            (HandShape.SCISSORS, HandShape.PAPER): 1, (HandShape.SCISSORS, HandShape.ROCK): -1, (HandShape.SCISSORS, HandShape.SCISSORS): 0
        }

    def get_payoff(self, my, opp):
        return self.rules.get((my, opp), 0)

    def solve_hybrid(self, my_hands, opp_hands, prediction=None):
        # 1. Offensive Mode: If we predict a move, counter it hard.
        if prediction is not None and prediction in opp_hands:
            score0 = self.get_payoff(my_hands[0], prediction)
            score1 = self.get_payoff(my_hands[1], prediction)
            if score0 > score1: return my_hands[0], f"KILLER: {my_hands[0].name}"
            elif score1 > score0: return my_hands[1], f"KILLER: {my_hands[1].name}"

        # 2. Defensive Mode: Nash Equilibrium
        # Calculate expected value of keeping Hand 0 vs Hand 1 against BOTH opponent options
        score0 = self.get_payoff(my_hands[0], opp_hands[0]) + self.get_payoff(my_hands[0], opp_hands[1])
        score1 = self.get_payoff(my_hands[1], opp_hands[0]) + self.get_payoff(my_hands[1], opp_hands[1])
        
        # Select the hand with the highest cumulative payoff/safety
        rec_hand = my_hands[0] if score0 >= score1 else my_hands[1]
        return rec_hand, f"NASH: {rec_hand.name}"

# --- 4. COMPUTER VISION ---
class HandDetector:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=True, 
            max_num_hands=4, 
            min_detection_confidence=0.5, 
            model_complexity=1
        )
        self.mp_draw = mp.solutions.drawing_utils

    def get_dist(self, p1, p2):
        return math.hypot(p1.x - p2.x, p1.y - p2.y)

    def classify_gesture(self, hand_landmarks):
        """
        Geometric classification based on finger extension.
        """
        # Compare tip to PIP joint distance vs wrist distance
        tips = [4, 8, 12, 16, 20]
        pips = [2, 6, 10, 14, 18]
        fingers_open = []
        
        # Thumb logic (different plane)
        fingers_open.append(
            self.get_dist(hand_landmarks.landmark[4], hand_landmarks.landmark[17]) > 
            self.get_dist(hand_landmarks.landmark[3], hand_landmarks.landmark[17])
        )

        # Other 4 fingers
        for i in range(1, 5):
            fingers_open.append(
                hand_landmarks.landmark[tips[i]].y < hand_landmarks.landmark[pips[i]].y
                if hand_landmarks.landmark[0].y > 0.5 else # Flip logic if hand is upside down? 
                # MediaPipe is normalized, so we stick to geometric distance relative to wrist
                self.get_dist(hand_landmarks.landmark[0], hand_landmarks.landmark[tips[i]]) >
                self.get_dist(hand_landmarks.landmark[0], hand_landmarks.landmark[pips[i]])
            )
            
        total_open = sum(fingers_open)
        
        # Classification Rules
        if total_open <= 1: return HandShape.ROCK
        if fingers_open[1] and fingers_open[2] and not fingers_open[3] and not fingers_open[4]: return HandShape.SCISSORS
        if total_open >= 4: return HandShape.PAPER
        return HandShape.UNKNOWN

    def process_static(self, image_path):
        img = cv2.imread(image_path)
        if img is None: return None, []
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.hands.process(img_rgb)
        detected = [] 
        
        if results.multi_hand_landmarks:
            for hand_lms in results.multi_hand_landmarks:
                # Use Average Y to separate players (Top vs Bottom)
                y_values = [lm.y for lm in hand_lms.landmark]
                avg_y = sum(y_values) / len(y_values)
                
                shape = self.classify_gesture(hand_lms)
                detected.append((shape, avg_y))
                
                # Visualization
                color = (0, 255, 0) if avg_y > 0.5 else (255, 0, 0)
                self.mp_draw.draw_landmarks(img_rgb, hand_lms, self.mp_hands.HAND_CONNECTIONS,
                                            self.mp_draw.DrawingSpec(color=color, thickness=2, circle_radius=2))
                                            
        return img_rgb, detected

## 3. Results & Verification

### 3.1 Instructor Verification Block
The following code block is designed for reproducibility. It initializes the `HandDetector` and `GameStrategy` classes defined above and iterates through the 5 sample images provided in the submission folder (`test1.jpg` through `test5.jpg`).

**How it works:**
1.  Loads the image.
2.  Detects all hands using MediaPipe.
3.  Sorts hands by Y-coordinate (Top = Opponent, Bottom = Player).
4.  Calculates the optimal move based on the identified hands.
5.  Displays the annotated image with the AI's decision.

In [ ]:
def test_static_image(image_path):
    detector = HandDetector()
    strategy = GameStrategy()
    
    print(f"\n--- Analyzing {image_path} ---")
    img_rgb, raw_data = detector.process_static(image_path)
    
    if img_rgb is None:
        print(f"File not found: {image_path}")
        return

    # Sort hands: Top (Opponent) vs Bottom (Player)
    # MediaPipe Y coordinates: 0.0 is Top, 1.0 is Bottom
    all_bottom = [d[0] for d in raw_data if d[1] > 0.5] 
    all_top = [d[0] for d in raw_data if d[1] <= 0.5]
    
    print(f" > Player Hands (Bottom): {[h.name for h in all_bottom]}")
    print(f" > Opponent Hands (Top):  {[h.name for h in all_top]}")

    if len(all_bottom) >= 1 and len(all_top) >= 1:
        # We need at least 1 hand per player to calculate a winner, 
        # ideally 2 for the full RPS-1 strategy.
        # This takes the first 2 hands found for each player.
        my_hand_selection = all_bottom[:2]
        opp_hand_selection = all_top[:2]
        
        # If only 1 hand is visible, duplicate it (assume forced move)
        if len(my_hand_selection) == 1: my_hand_selection.append(my_hand_selection[0])
        if len(opp_hand_selection) == 1: opp_hand_selection.append(opp_hand_selection[0])
            
        rec_hand, logic = strategy.solve_hybrid(my_hand_selection, opp_hand_selection)
        
        print(f" > Strategy Engine: {logic}")
        
        # Annotation
        h, w, _ = img_rgb.shape
        cv2.rectangle(img_rgb, (0, h-60), (w, h), (0,0,0), -1)
        cv2.putText(img_rgb, f"AI: {logic}", (20, h-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        print(" > Status: Insufficient hands detected for gameplay analysis.")

    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# --- EXECUTION LOOP ---
# Ensure test1.jpg, test2.jpg... are in the same folder
sample_images = ['test1.jpg', 'test2.jpg', 'test3.jpg', 'test4.jpg', 'test5.jpg']

for img_file in sample_images:
    test_static_image(img_file)

## 4. Discussion

### 4.1 Design Decisions
* **MediaPipe vs. YOLO:** We initially considered YOLO for object detection. However, RPS relies on the *state* of the fingers (open/closed) rather than just the object class. Training a YOLO model to distinguish between "Rock" and "Paper" requires a massive, varied dataset. MediaPipe's skeletal landmarking provided a geometric solution that was robust, lightweight, and required no training data.
* **The "Hybrid" Notebook:** To ensure reproducibility for grading (as requested), we extracted the core logic classes from our Flask web application (`app.py`) and embedded them directly into this notebook. This allows the instructor to verify the game logic without the complexity of setting up a local web server environment.

### 4.2 Challenges & Mitigation
* **Lighting & Occlusion:** Computer vision struggles in poor lighting. To mitigate this, we implemented a confidence threshold in `HandDetector` and visual feedback (Red/Green skeletons) so the user knows immediately if tracking is lost.
* **Game State Synchronization:** In the real-time app, syncing the AI's calculation with the exact moment players reveal their hands was difficult. We solved this by implementing a `GameState` state machine (IDLE -> COUNTDOWN -> SHOOT) to lock in predictions only at the critical moment.

## 5. Conclusion
The RPS-Neural-Link project successfully demonstrates a Cyber-Physical System where physical input (gestures) drives cyber analysis (Game Theory), resulting in physical augmentation (AR feedback). The system correctly identifies hand states and applies Nash Equilibrium to provide optimal advice, satisfying the core learning objectives of the course.

---

### 6. References
1.  **MediaPipe:** Lugaresi, C., et al. "MediaPipe: A Framework for Building Perception Pipelines." *arXiv preprint arXiv:1906.08172* (2019).
2.  **Flask:** Grinberg, M. "Flask Web Development: Developing Web Applications with Python." *O'Reilly Media, Inc.* (2018).
3.  **Nash Equilibrium:** Nash, J. "Non-cooperative games." *Annals of mathematics* (1951): 286-295.

---

### Appendix: Web Application
In addition to this notebook, the submitted zip file contains a full **Flask Web Application** (`app.py`, `templates/`, `static/`). This app provides a fully immersive AR experience with a cyberpunk HUD, live webcam feed, and real-time tournament tracking. A video demonstration of this app is included in the submission.